# Clasificación de documentos con word2vec
### Dpto. Ciencias de la Computación e Inteligencia Artificial - Universidad de Sevilla
Realizado por Lucas Antoñanzas del Villar (lucantdel) y Jaime García García (jaigagar1)

## Construcción del corpus
En este apartado, se describe el proceso de construcción del corpus utilizado en nuestro proyecto de clasificación de documentos. Encontramos una [compilación de 40 datasets de noticias gratis](https://www.linkedin.com/pulse/free-news-datasets-mega-compilation-rajat-thakur/) y tras contemplar varias opciones escogimos una [colección de textos periodísticos en inglés de la BBC](http://mlg.ucd.ie/datasets/bbc.html). Esta fuente nos proporciona una variedad de artículos de diferentes categorías, en concreto: business, entertainment, politics, sport y tech.

Para obtener el corpus, accedimos al enlace proporcionado y descargamos los archivos de texto agrupados por categorías en diferentes carpetas. Cada carpeta contiene múltiples archivos .txt de una categoría específica, y cada uno de estos, un artículo (título, subtítulo y cuerpo de la noticia).

Una vez obtenidos los textos de las diferentes categorías, procedimos a transformar todos estos archivos .txt a un solo archivo .csv con las siguientes columnas: id, article y category. Etiquetamos los artículos según la carpeta a la que pertenecen. 

In [2]:
import os
import csv

# Directorio raíz donde se encuentran las carpetas por categoría
root_directory = "./bbc-fulltext/bbc"

# Nombre del archivo .csv de salida
csv_file = "./news.csv"

# Lista para almacenar las filas del archivo .csv
csv_rows = []

# Número por el que comenzarán los id
id = 0

# Recorre todas las carpetas en el directorio raíz
for category in os.listdir(root_directory):
    # Obtiene la ruta completa de la carpeta de la categoría
    category_folder  = os.path.join(root_directory, category)
    
    # Verifica que sea una carpeta
    if os.path.isdir(category_folder):
        # Recorre todos los archivos .txt en la carpeta de la categoría
        for txt_file  in os.listdir(category_folder):
            # Obtiene la ruta completa del archivo .txt
            txt_file_path  = os.path.join(category_folder, txt_file)
            
            # Lee el contenido del archivo .txt
            with open(txt_file_path, 'r') as file:
                content  = file.read().replace(".\n\n", ". ").replace("\n\n", ". ").replace("\n", "")
                
                # Agrega una nueva fila al archivo .csv
                row = [id, category, content]
                csv_rows.append(row)
                id += 1

# Escribe las filas en el archivo .csv
with open(csv_file, 'w', newline='') as file:
    csv_writer  = csv.writer(file, delimiter=';')
    csv_writer.writerow(['id', 'category', 'content'])
    csv_writer.writerows(csv_rows)

print("Successfully generated the .csv file.")

Successfully generated the .csv file.


#### Análisis del corpus

In [3]:
import pandas as pd

df = pd.read_csv("news.csv", sep=';')

print(df['category'].value_counts())

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64


In [4]:
print("Número total de filas:", df.shape[0])

Número total de filas: 2225


A continuación se muestran 5 filas aleatorias del archivo .csv que hemos generado:

In [5]:
import random

random_index = random.sample(range(0, df.shape[0]), 5)
df.iloc[random_index]

,id,category,content
12,12,business,Peugeot deal boosts Mitsubishi. Struggling Jap...
1854,1854,tech,Solutions to net security fears. Fake bank e-m...
227,227,business,Tsunami 'to hit Sri Lanka banks'. Sri Lanka's ...
243,243,business,Market unfazed by Aurora setback. As the Auror...
541,541,entertainment,Redford's vision of Sundance. Despite sporting...


## Tokenización de texto y eliminación de 'stopwords'

In [7]:
from nltk.corpus import stopwords
import re

def tokenize_and_remove_stopwords(text):
    # Elimina todo lo que no sean letras y lo sustituye por espacios
    review_text = re.sub("[^a-zA-Z]"," ", text)
    
    # Pasa las palabras a minúsculas y separa las palabras
    tokens = review_text.lower().split()
    
    # Elimina los caracteres sueltos
    tokens = [token for token in tokens if len(token) > 1]
    
    # Elimina las stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

A continuación, mostraremos un articulo del csv y el resultado de tokenizarlo y eliminar las stopwords:

In [8]:
article = df['content'].tolist()[1]
print(article)

Dollar gains on Greenspan speech. The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise. And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan's speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. "I think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time," said Robert Sinche, head of currency strategy at Bank of America in New York. "He's taking a longer-term view, laying out a set of conditions under which 

In [10]:
import nltk
nltk.download('stopwords')

print(tokenize_and_remove_stopwords(article))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...


['dollar', 'gains', 'greenspan', 'speech', 'dollar', 'hit', 'highest', 'level', 'euro', 'almost', 'three', 'months', 'federal', 'reserve', 'head', 'said', 'us', 'trade', 'deficit', 'set', 'stabilise', 'alan', 'greenspan', 'highlighted', 'us', 'government', 'willingness', 'curb', 'spending', 'rising', 'household', 'savings', 'factors', 'may', 'help', 'reduce', 'late', 'trading', 'new', 'york', 'dollar', 'reached', 'euro', 'thursday', 'market', 'concerns', 'deficit', 'hit', 'greenback', 'recent', 'months', 'friday', 'federal', 'reserve', 'chairman', 'mr', 'greenspan', 'speech', 'london', 'ahead', 'meeting', 'finance', 'ministers', 'sent', 'dollar', 'higher', 'earlier', 'tumbled', 'back', 'worse', 'expected', 'us', 'jobs', 'data', 'think', 'chairman', 'taking', 'much', 'sanguine', 'view', 'current', 'account', 'deficit', 'taken', 'time', 'said', 'robert', 'sinche', 'head', 'currency', 'strategy', 'bank', 'america', 'new', 'york', 'taking', 'longer', 'term', 'view', 'laying', 'set', 'condi

[nltk_data]   Unzipping corpora\stopwords.zip.


## Stemming con NTLK

In [12]:
from nltk.stem import SnowballStemmer

# Definir el stemmer a utilizar
stemmer = SnowballStemmer('english')  # Utiliza SnowballStemmer para español, puedes cambiarlo según el idioma

def preprocess_text(text):
    # Tokenización: utilizamos la función creada en el apartado anterior
    tokens = tokenize_and_remove_stopwords(text)
    
    # Aplicar stemming
    processed_tokens = [stemmer.stem(token) for token in tokens]
    
    return processed_tokens

Este es el resultado de aplicar stemming a los tokens del articulo de ejemplo del apartado anterior:

In [13]:
print(preprocess_text(article))

['dollar', 'gain', 'greenspan', 'speech', 'dollar', 'hit', 'highest', 'level', 'euro', 'almost', 'three', 'month', 'feder', 'reserv', 'head', 'said', 'us', 'trade', 'deficit', 'set', 'stabilis', 'alan', 'greenspan', 'highlight', 'us', 'govern', 'willing', 'curb', 'spend', 'rise', 'household', 'save', 'factor', 'may', 'help', 'reduc', 'late', 'trade', 'new', 'york', 'dollar', 'reach', 'euro', 'thursday', 'market', 'concern', 'deficit', 'hit', 'greenback', 'recent', 'month', 'friday', 'feder', 'reserv', 'chairman', 'mr', 'greenspan', 'speech', 'london', 'ahead', 'meet', 'financ', 'minist', 'sent', 'dollar', 'higher', 'earlier', 'tumbl', 'back', 'wors', 'expect', 'us', 'job', 'data', 'think', 'chairman', 'take', 'much', 'sanguin', 'view', 'current', 'account', 'deficit', 'taken', 'time', 'said', 'robert', 'sinch', 'head', 'currenc', 'strategi', 'bank', 'america', 'new', 'york', 'take', 'longer', 'term', 'view', 'lay', 'set', 'condit', 'current', 'account', 'deficit', 'improv', 'year', 'ne

A continuación, crearemos el vocabulario final en función de la frecuencia de las palabras que aparecen en el corpus preprocesado. Podemos establecer un umbral para filtrar las palabras de baja frecuencia y considerar solo aquellas que aparecen con mayor regularidad. Esta selección nos permitirá reducir la dimensionalidad y centrarnos en las palabras más informativas para la clasificación de documentos.

In [15]:
from collections import Counter

# Definir un umbral para filtrar palabras de baja frecuencia
threshold = 10

# Creación del corpus y vocabulario de todos los artículos
corpus = [preprocess_text(text) for text in df['content'].tolist()]
all_words = [word for doc in corpus for word in doc]
all_words_freq = Counter(all_words)
vocabulary = [word for word, freq in all_words_freq.items() if freq >= threshold]

## Word2Vec
Word2Vec es un algoritmo de aprendizaje automático utilizado para representar palabras como vectores numéricos en un espacio vectorial. Se basa en la idea de que el significado de una palabra se puede capturar en función de su contexto lingüístico. Word2Vec utiliza una red neuronal para aprender a predecir la probabilidad de que una palabra aparezca en función de las palabras vecinas en un corpus de texto grande. A medida que el modelo se entrena en un corpus, las palabras similares tienden a tener vectores similares en el espacio vectorial. Estos vectores resultantes pueden usarse para medir la similitud entre palabras, realizar operaciones de analogía y mejorar el rendimiento en tareas de procesamiento de lenguaje natural.

#### Creación del modelo

In [17]:
import multiprocessing
import gensim
from gensim.models import Word2Vec

# Cuenta el número de núcleos de tu ordenador
cores = multiprocessing.cpu_count() 

w2v_model = Word2Vec(min_count=threshold, # min_count = int - Ignora todas las palabras con una frecuencia absoluta total menor que esto - (2, 100)
                 window=5, # window = int - La distancia máxima entre la palabra actual y la palabra predicha dentro de una oración. Por ejemplo, "window" palabras a la izquierda y "window" palabras a la derecha de nuestro objetivo - (2, 10)
                 vector_size=200, # size = int - Dimensionalidad de los vectores de características - (50, 300)
                 sample=1e-4, # sample = float - Umbral para configurar qué palabras de alta frecuencia se reducen aleatoriamente. Muy influyente - (0, 1e-5)
                 alpha=0.03, # alpha = float - Tasa de aprendizaje inicial - (0.01, 0.05)
                 min_alpha=0.0007, # min_alpha = float - La tasa de aprendizaje disminuirá linealmente hasta min_alpha a medida que avance el entrenamiento. Para establecerlo: alpha - (min_alpha * epochs) ~ 0.00
                 negative=10, # negative = int - Si es > 0, se utilizará muestreo negativo, donde el entero para negative especifica cuántas "palabras de ruido" se deben seleccionar. Si se establece en 0, no se utiliza muestreo negativo - (5, 20)
                 workers=cores-1) # workers = int - Utiliza esta cantidad de hilos de trabajo para entrenar el modelo (entrenamiento más rápido con máquinas multicore)

#### Construcción del vocabulario

In [19]:
w2v_model.build_vocab(corpus)

#### Entrenamiento del modelo

In [20]:
w2v_model.train(corpus, 
            total_examples=w2v_model.corpus_count, # total_examples = int - Contador de oraciones
            epochs=20) # epochs = int - Número de iteraciones (épocas) sobre el corpus - [10, 20, 30]

(5645065, 9692000)

#### Ejemplos
WARNING: Las palabras introducidas como parámetro deben estar contenidas en el corpus

In [21]:
# Obtener el vector de una palabra
w2v_model.wv['player']

array([-1.21677113e+00,  2.11862177e-01, -3.19521219e-01, -8.34378749e-02,
       -3.13301951e-01, -4.79111284e-01,  3.40746760e-01, -5.43864481e-02,
        8.36227648e-03,  8.15962628e-02, -8.82159472e-02, -6.98183417e-01,
       -7.97175050e-01,  6.38369858e-01, -4.90571558e-01, -5.69918931e-01,
        1.65025249e-01,  7.18296528e-01,  7.53071979e-02, -5.83105445e-01,
        3.20539117e-01, -3.32210332e-01,  6.31354809e-01,  6.21958494e-01,
        2.38747876e-02, -4.13469225e-01,  6.17894530e-01, -1.67619616e-01,
       -7.81229258e-01,  5.28283007e-02,  2.05473527e-01,  6.07173264e-01,
        1.40435547e-01,  2.48686999e-01,  3.79380673e-01, -3.75763059e-01,
        9.12328899e-01,  4.53429818e-01,  8.28667581e-01,  3.80425632e-01,
       -8.82428791e-03,  2.80631837e-02, -3.22194934e-01,  4.64758158e-01,
       -6.29261509e-02, -1.36482030e-01, -7.50863075e-01, -3.22237432e-01,
       -6.57721400e-01,  3.04776758e-01, -1.06045529e-01,  2.26440355e-01,
       -1.13319564e+00, -

In [22]:
# Buscar palabras similares
w2v_model.wv.most_similar('skill')

[('disciplin', 0.8215752840042114),
 ('teach', 0.7981538772583008),
 ('pupil', 0.770003616809845),
 ('educ', 0.7632160186767578),
 ('citizenship', 0.7582079768180847),
 ('nhs', 0.7455408573150635),
 ('social', 0.7404744029045105),
 ('disadvantag', 0.7298678755760193),
 ('lesson', 0.7138912677764893),
 ('democraci', 0.7130954265594482)]

In [23]:
# Encontrar la palabra que no encaja en el conjunto
w2v_model.wv.doesnt_match(['market', 'player', 'bank'])

'player'

## JUNIO: Clasificación de documentos
### Naive-Bayes Multinomial

In [24]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# Preparación de los datos
count_vec = CountVectorizer()
bow = count_vec.fit_transform(df['content'])
bow = np.array(bow.todense())

X = bow
y = df['category']

# Creación del conjunto de características
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y) 

# Entrenamiento del clasificador MultinomialNB
mnb_classifier = MultinomialNB()
model = mnb_classifier.fit(X_train, y_train)

# Evaluación del clasificador
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

NameError: name 'classification_report' is not defined

#### Matriz de confusión

In [ ]:
import matplotlib.pyplot as plt

confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

labels = set(df['category'].tolist())

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = labels)
cm_display.plot(cmap='Oranges')

plt.xticks(rotation=90)
plt.show()

### Word2Vec

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Preparación de los datos
X = df['content']  # Textos
y = df['category']  # Etiquetas

# Representación de los textos utilizando Word2Vec
vector_size = w2v_model.vector_size

def text_to_vector(text):
    words = preprocess_text(text)
    vectors = []
    for word in words:
        if word in w2v_model.wv:
            vectors.append(w2v_model.wv[word])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)
    
# Creación del conjunto de características
X = np.array([text_to_vector(text) for text in X])
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Entrenamiento del clasificador RandomForest
rf_classifier = RandomForestClassifier(n_estimators = 100) # n_estimators = int - Representa el número de árboles de decisión que se utilizarán en el conjunto de bosques aleatorios.
rf_classifier.fit(X_train, y_train)

# Evaluación del clasificador
y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

#### Matriz de confusión

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

labels = set(df['category'].tolist())

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = labels)
cm_display.plot(cmap='Oranges')

plt.xticks(rotation=90)
plt.show()